In [1]:
%run ../../multiplicity/src/binning.py
%run ../../multiplicity/src/plot.py
ROOT.EnableImplicitMT(8)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-8v5dukqu because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to JupyROOT 6.22/06


In [2]:
def get_tfile_ttree(pion_pair,version):
    if version == "data":
        if pion_pair == [211,111]:
            filesuffix = "Fall2018_RGA_inbending_merged_cuts.root"
            #filesuffix = "nSidis_RGA_5036.root"
        elif pion_pair == [-211,111]:
            filesuffix = "Spring2019_RGA_outbending_merged_cuts.root"
    elif version == "MC":
        if pion_pair == [211,111]:
            filesuffix = "MC_RGA_inbending_merged_cuts.root"
            #filesuffix = "MC_RGA_3051_0.root"
        elif pion_pair == [-211,111]:
            filesuffix = "MC_RGA_outbending_merged_cuts.root"

    pid_str_1 = str(pion_pair[0]).replace("-211","piminus").replace("211","piplus").replace("111","pi0")
    pid_str_2 = str(pion_pair[1]).replace("-211","piminus").replace("211","piplus").replace("111","pi0")

    dihadron_str = f"{pid_str_1}_{pid_str_2}"

    tfile = ROOT.TFile(f"/volatile/clas12/users/gmat/clas12analysis.sidis.data/clas12_dihadrons/projects/pipi0_paper_RGA_only/data/{dihadron_str}/{filesuffix}")
    ttree = tfile.Get("dihadron_cuts")
    ttree.SetAlias("rec_passDihadron","x>0")
    ttree.SetAlias("rec_x","x")
    ttree.SetAlias("rec_Q2","Q2")
    return tfile,ttree

In [3]:
pion_pair=[211,111]
tfile_data,tdata = get_tfile_ttree(pion_pair,"data")
tfile_mc,tmc = get_tfile_ttree(pion_pair,"MC")

In [4]:
#######################################################################
# 2d Rectangular Binning and default Custom Binning
#######################################################################

rect_bin_factory = RectBinFactory()
rect_bin_factory.make_bins("th_21",np.linspace(0.1,0.6,6))
rect_bin_factory.make_bins("th_22",np.linspace(0.1,0.6,6))

custom_bin_factory = CustomBinFactory(["x","Q2"])

custom_bin_factory.add_curve("low_Q2","Q2>0")
custom_bin_factory.add_curve("high_Q2","Q2<10")


custom_bin_factory.make_bin(["low_Q2","high_Q2"],"x>0","x<1")

In [5]:
#######################################################################
# Load all Binnings into the BinManager
#######################################################################

bin_manager = BinManager()
bin_manager.load_factory(rect_bin_factory)
bin_manager.load_factory(custom_bin_factory)

In [6]:
bin_manager.has_custom_bins=False

In [7]:
histograms=make_multiple_bin_histos(bin_manager, tdata, tmc, 
                         [#(";x;Q^{2} [GeV^{2}]",100,0,1,100,1,10),
                         (";#theta(#gamma1);#theta(#gamma2)",100,0.05,0.65,100,0.05,0.65),
                         (";#phi_{h};Counts",100,-3.3,3.3)],
                                    plot_variable="phi_h",
                                    extra_cut="x>0",
                                   do_full_tcut=False)

In [10]:
import os
from ROOT import TCanvas

latex_template = r"""
\documentclass[12pt]{article}
\usepackage{graphicx}
\usepackage[margin=0.5in]{geometry} 
\begin{document}
%s
\end{document}
"""

figure_template = r"""
\begin{figure}[h]
\centering
\includegraphics[width=0.95\textwidth]{%s}
\caption{Plot Number = %s}
\end{figure}
"""

# Create a PDF for each canvas and write them to a LaTeX document
figures = ""
for m in range(1, 26):
    canvas = plot_map(bin_manager, histograms, tdata, tmc, m)#,scale_same=True)
    canvas.SaveAs(f"tmp_plots/plot_{m}.pdf")  # Save TCanvas as PDF
    figures += figure_template % (f"tmp_plots/plot_{m}.pdf", str(m))

latex_doc = latex_template % figures

# Write LaTeX document to a file
with open("data_v_monte_carlo_phi_h.tex", 'w') as f:
    f.write(latex_doc)

Error in <TList::Clear>: A list is accessing an object (0x55b8f71cf040) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b9008a9650) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b90572c930) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b8f2367330) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b84b16cc90) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b8fe1ff870) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b8f93528e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b8f4a69ea0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x55b8dc7c8dc0) already deleted (list name = TList)
Error in <TList::Clear>: A l

In [11]:
# Compile LaTeX file to PDF
os.system("pdflatex data_v_monte_carlo_phi_h.tex")

0

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./data_v_monte_carlo_phi_h.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size12.clo))
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/trig.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-cfg/graphics.cfg)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-def/pdftex.def)))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifte